In [4]:
using Plots,Random,Statistics,DataStructures
plotlyjs()
using BenchmarkTools

Defining the geometry and materials of Reed's Cell

In [58]:
widths=[2,1,2,1,2]
NΔx=[1,1,1,1,1]*20

n=widths,NΔx ;
#Defining Geometry
function geometry(x,widths=widths)
    i=0
    while x>=0
        i+=1
        x-=widths[i]

    end
    y=map(Bool,widths*0)
    y[i]=true
    return y
end

#assigning material properties to the geometry
Σt(x)=sum(geometry(x).*[50.0 , 5.0 , 0 , 1.0 , 1.0])/sum(geometry(x)); #total cross section
Σr(x)=sum(geometry(x).*[50.0 , 5.0 , 0 , 0.1 , 0.1])/sum(geometry(x)); #removal cross section
Σs(x)=Σt(x)-Σr(x)
Q(x) =sum(geometry(x).*[50.0 , 0.0 , 0 , 1.0 , 0.0])/sum(geometry(x)); #source term
D(x) =1/3/Σt(x); #Diffusion Coefficient

f=D,Σr,Q 

#Albedo Boundary Conditions
β1=1  #Reflective
β2=0   #Vacuum

#Partial currents
J1=0   #left
J2=0 ; #right

bc=β1,β2,J1,J2 ;

Flux computation without any variance reduction   
ϕ  - flux from collision rate   
ϕ2 - flux from track length   
ϕ3 - flux from scattering rate   
ϕ4 - flux from absorption rate   

In [59]:
Δx=Float64[]
for i=1:length(widths)
    Δx=[Δx;ones(NΔx[i]).*widths[i]/NΔx[i]]
end
I=sum(NΔx)
x=[sum(Δx[1:i])-*(Δx[i]/2) for i=1:I]
Reactions=zeros(I);
Abs=copy(Reactions)
L1,L2=0,0;

Σ_t=map(Σt,x)
Σ_r=map(Σr,x)
Σ_s=map(Σs,x)
L=Σ_t.*Δx
Ls=Σ_s.*Δx
Lr=Σ_r.*Δx
Qp=reduce(vcat,[Q(x[i])*Δx[i] for i=1:I])
Sf=sum(Qp)
Qp/=Sf
ϕ=zeros(I);
ϕ2=zeros(I)
φ=copy(ϕ);
φ2=copy(ϕ);
φ3=copy(ϕ);
φ4=copy(ϕ);
Batches=100
tally1=zeros(Batches)
@time for batch=1:Batches
    NN=100000
    ϕ2*=0
    for j=1:NN
        pQ=rand()

        i=0
        while pQ>0
            i+=1
            pQ-=Qp[i]    
        end

        xQ=x[i]+(0.5+pQ/Qp[i])*Δx[i] 

        br=false
        while true
            μ=2*(rand()-0.5)
            invμ=1/abs(μ)
            l=-log(rand())
            if μ>0
                ΔxQ=x[i]+0.5*Δx[i]-xQ
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ
                shift=1
            elseif μ<0
                ΔxQ=xQ-x[i]+0.5*Δx[i]
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ
                shift=-1
            end
            while l>0
                if i==1 && μ<0 
                    if β1>rand()
                        μ*=-1
                        i=0
                        shift=1
                    else
                        br=true
                        break
                    end  
                elseif i==I && μ>0
                    if β2>rand()
                        μ*=-1
                        i=I+1
                        shift=-1
                    else
                        br=true
                        break
                    end
                end

                i+=shift
                l-=L[i]*invμ
                ϕ2[i]+=invμ*Δx[i]

            end
            if br
                break
            end

            Reactions[i]+=1 
            if μ>0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]+0.5*Δx[i]+ΔxQ
                ϕ2[i]+=invμ*ΔxQ
            elseif μ<0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]-0.5*Δx[i]+ΔxQ
                ϕ2[i]-=invμ*ΔxQ
            end 
            if Σ_t[i]*rand()<Σ_r[i]
                Abs[i]+=1
                break
            end
        end
    end
    ϕ=Sf*Reactions./L/NN
    Scats=Reactions-Abs
    ϕ3=Sf*Scats./Ls/NN
    ϕ4=Sf*Abs./Lr/NN
    Reactions*=0
    Abs*=0
    tally1[batch]=ϕ[72]
    φ+=ϕ
    ϕ2*=Sf/NN
    ϕ2./=Δx
    φ2+=ϕ2
    φ3+=ϕ3
    φ4+=ϕ4
end
plot(x,[φ,φ2,φ3,φ4]/Batches)

 28.371620 seconds (696.84 M allocations: 10.385 GiB, 4.44% gc time)


Flux calculation with variance reduction, the weight reduces exponentially   
ϕ  - flux from absorption rate/tracklength   
ϕ2 - flux from scattering rate   
ϕ3 - flux from total collision rate  

In [60]:
Δx=Float64[]
for i=1:length(widths)
    Δx=[Δx;ones(NΔx[i]).*widths[i]/NΔx[i]]
end
I=sum(NΔx)
x=[sum(Δx[1:i])-*(Δx[i]/2) for i=1:I]
Reactions=zeros(I);
Abs=copy(Reactions)
L1,L2=0,0;

Σ_t=map(Σt,x)
Σ_r=map(Σr,x)
Σ_s=map(Σs,x)
L=Σ_t.*Δx
Ls=Σ_s.*Δx
Lr=Σ_r.*Δx
Qp=reduce(vcat,[Q(x[i])*Δx[i] for i=1:I])
Sf=sum(Qp)
Qp/=Sf
ϕ=zeros(I);
ϕ2=zeros(I)
ϕ3=zeros(I)
φ=copy(ϕ);
φ2=copy(ϕ);
φ3=copy(ϕ);
Batches=100
tally2=zeros(Batches)
@time for batch=1:Batches
    #print("\nBatch: ", batch)
    NN=100000
    ϕ=zeros(I);
    for j=1:NN
        
        pQ=rand()

        i=0
        while pQ>0
            i+=1
            pQ-=Qp[i]    
        end

        xQ=x[i]+(0.5+pQ/Qp[i])*Δx[i]
        ω=1

        br=false
        while ω>1e-38
            if ω<0.25
                if rand()<ω
                    ω=1
                    #print("I am alive xD")
                else
                    ω=0
                    #print("Save me I am dying!\n")
                    continue
                end
            end
            μ=2*(rand()-0.5)
            invμ=1/abs(μ)
            l=-log(rand())
                
            if μ>0
                ΔxQ=x[i]+0.5*Δx[i]-xQ
                l-=ΔxQ*Σ_s[i]*invμ
                
                if l>0
                    Expa=exp(-ΔxQ*Σ_r[i]*invμ)
                    ϕ[i]+=ω*(1-Expa)
                    ω*=Expa
                    shift=1

                else
                    l+=ΔxQ*Σ_s[i]*invμ
                    Expa=exp(-l*Σ_r[i]/Σ_s[i])
                    ϕ[i]+=ω*(1-Expa)
                    ω*=Expa
                    Reactions[i]+=ω
                    xQ+=l/Σ_s[i]/invμ
                    continue
                end
                
            elseif μ<0
                ΔxQ=xQ-x[i]+0.5*Δx[i]
                l-=ΔxQ*Σ_s[i]*invμ
                    
                if l>0    
                    Expa=exp(-ΔxQ*Σ_r[i]*invμ)
                    ϕ[i]+=ω*(1-Expa)
                    ω*=Expa
                    shift=-1
                else
                    l+=ΔxQ*Σ_s[i]*invμ
                    Expa=exp(-l*Σ_r[i]/Σ_s[i])
                    ϕ[i]+=ω*(1-Expa)
                    ω*=Expa
                    Reactions[i]+=ω
                    xQ-=l/Σ_s[i]/invμ
                    continue
                end
            end
            
            while l>0 && ω>1e-38         

                if i==1 && μ<0 
                    if β1>rand()
                        μ*=-1
                        i=0
                        shift=1
                    else
                        br=true
                        break
                    end  
                elseif i==I && μ>0
                    if β2>rand()
                        μ*=-1
                        i=I+1
                        shift=-1
                    else
                        br=true
                        break
                    end
                end
                i+=shift
                l-=Ls[i]*invμ
                Expa=exp(-Lr[i]*invμ)
                ϕ[i]+=ω*(1-Expa)
                ω*=Expa

            end
            
            if br || ω<1e-38 
                break
            end

            ω/=Expa
            ϕ[i]-=ω*(1-Expa)
            if μ>0               
                xQ=x[i]+0.5*Δx[i]+(l*μ/Σ_s[i])
                ΔxQ=xQ-x[i]+0.5*Δx[i]
                Expa=exp(-ΔxQ*Σ_r[i]*invμ)
                ω*=Expa
                ϕ[i]+=ω*(1-Expa)
            elseif μ<0
                xQ=x[i]-0.5*Δx[i]+l*μ/Σ_s[i]
                ΔxQ=x[i]+0.5*Δx[i]-xQ
                Expa=exp(-ΔxQ*Σ_r[i]*invμ)
                ω*=Expa
                ϕ[i]+=ω*(1-Expa)
            end
            Reactions[i]+= ω

        end
    end
    ϕ2=Sf*Reactions./Ls/NN
    Reactions+=ϕ
    ϕ./=Lr
    ϕ*=Sf/NN
    tally2[batch]=ϕ[72]
    φ+=ϕ
    ϕ3=Sf*Reactions./L/NN
    φ2+=ϕ2
    φ3+=ϕ3
    Reactions*=0
end
plot(x,[φ,φ2,φ3]/Batches)

132.004772 seconds (3.17 G allocations: 47.169 GiB, 4.41% gc time)


Flux calculation with variance reduction, the weight decreases fractionally after each collision  
ϕ  - flux from collision rate   
ϕ2 - flux from track length   
ϕ3 - flux from scattering rate   
ϕ4 - flux from absorption rate

In [61]:
Δx=Float64[]
for i=1:length(widths)
    Δx=[Δx;ones(NΔx[i]).*widths[i]/NΔx[i]]
end
I=sum(NΔx)
x=[sum(Δx[1:i])-*(Δx[i]/2) for i=1:I]
Reactions=zeros(I);
Abs=copy(Reactions)
L1,L2=0,0;

Σ_t=map(Σt,x)
Σ_r=map(Σr,x)
Σ_s=map(Σs,x)
L=Σ_t.*Δx
Ls=Σ_s.*Δx
Lr=Σ_r.*Δx
Pa=Σ_r./Σ_t
Qp=reduce(vcat,[Q(x[i])*Δx[i] for i=1:I])
Sf=sum(Qp)
Qp/=Sf
ϕ=zeros(I);
ϕ2=zeros(I)
φ=copy(ϕ);
φ2=copy(ϕ);
φ3=copy(ϕ);
φ4=copy(ϕ);
Batches=100
tally3=zeros(Batches)
@time for batch=1:Batches
    NN=100000
    ϕ2*=0
    for j=1:NN
        pQ=rand()

        i=0
        while pQ>0
            i+=1
            pQ-=Qp[i]    
        end

        xQ=x[i]+(0.5+pQ/Qp[i])*Δx[i] 
        ω=1
        br=false
        while ω>1e-38
            if ω<0.25
                if rand()<ω
                    ω=1
                    #print("I am alive xD")
                else
                    ω=0
                    #print("Save me I am dying!\n")
                    continue
                end
            end
            μ=2*(rand()-0.5)
            invμ=1/abs(μ)
            l=-log(rand())
                
            if μ>0
                ΔxQ=x[i]+0.5*Δx[i]-xQ
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ*ω
                shift=1
            elseif μ<0
                ΔxQ=xQ-x[i]+0.5*Δx[i]
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ*ω
                shift=-1
            end
            while l>0
                if i==1 && μ<0 
                    if β1>rand()
                        μ*=-1
                        i=0
                        shift=1
                    else
                        br=true
                        break
                    end  
                elseif i==I && μ>0
                    if β2>rand()
                        μ*=-1
                        i=I+1
                        shift=-1
                    else
                        br=true
                        break
                    end
                end

                i+=shift
                l-=L[i]*invμ
                ϕ2[i]+=invμ*Δx[i]*ω

            end
            if br
                break
            end

            
            if μ>0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]+0.5*Δx[i]+ΔxQ
                ϕ2[i]+=invμ*ΔxQ*ω
            elseif μ<0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]-0.5*Δx[i]+ΔxQ
                ϕ2[i]-=invμ*ΔxQ*ω
            end
            
            Abs[i]+=ω*Pa[i]
            ω*=1-Pa[i]
            Reactions[i]+=ω
        end
    end
    ϕ=Sf*(Reactions+Abs)./L/NN
    ϕ3=Sf*Reactions./Ls/NN
    ϕ4=Sf*Abs./Lr/NN
    Reactions*=0
    Abs*=0
    tally3[batch]=ϕ[72]
    ϕ2*=Sf/NN
    ϕ2./=Δx
    φ+=ϕ
    φ2+=ϕ2
    φ3+=ϕ3
    φ4+=ϕ4
end
plot(x,[φ,φ2,φ3,φ4]/Batches)

 30.626059 seconds (725.39 M allocations: 10.811 GiB, 4.33% gc time)


In [62]:
mean(tally1),std(tally1)/ sqrt(Batches)

(1.9391999999999998, 0.022425869081050213)

In [63]:
mean(tally2),std(tally2)/ sqrt(Batches)

(1.962417044433915, 0.010697159770498892)

In [64]:
mean(tally3),std(tally3)/ sqrt(Batches)

(1.9353016137089398, 0.022166017165895745)

In [65]:
plot(tally1)
plot!(tally2)
plot!(tally3)

In [66]:
plot(x,ϕ)

Problem description for an eigenvalue problem

In [70]:
Σs(x)=0.8

Σt(x)=1.0

νΣf(x)=1.0

Σr(x)=0.2

β1=0   #Reflective
β2=0   #Vacuum

widths=[8]
NΔx=[100];

Eigenvalue calculation without survival biasing

In [71]:
Δx=Float64[]
for i=1:length(widths)
    Δx=[Δx;ones(NΔx[i]).*widths[i]/NΔx[i]]
end
I=sum(NΔx)
x=[sum(Δx[1:i])-*(Δx[i]/2) for i=1:I]
Reactions=zeros(I);
Abs=copy(Reactions)
L1,L2=0,0;
νΣ_f=reduce(vcat,[νΣf(x[i]) for i=1:I])
Σ_t=reduce(vcat,[Σt(x[i]) for i=1:I])
Σ_r=reduce(vcat,[Σr(x[i]) for i=1:I])
Σ_s=reduce(vcat,[Σs(x[i]) for i=1:I])
L=reduce(vcat,[Σt(x[i])*Δx[i] for i=1:I]);
Ls=reduce(vcat,[Σs(x[i])*Δx[i] for i=1:I])
Lr=reduce(vcat,[Σr(x[i])*Δx[i] for i=1:I])
ϕ=ones(I);
ϕ2=zeros(I)
Kp=1
Fp=reduce(vcat,[νΣ_f[i]*Δx[i]*ϕ[i] for i=1:I])/Kp
Sf=sum(Fp)
Qp=Fp/Sf;
Pa=Σ_r./Σ_t

Batches=100
inactive=20
K_vec=zeros(Batches);
@time for batch=1:Batches
    ϕ2*=0
    Particles=300000
    for particle=1:Particles
        pQ=rand()

        i=0
        while pQ>0
            i+=1
            pQ-=Qp[i]    
        end

        xQ=x[i]+(0.5+pQ/Qp[i])*Δx[i] 

        br=false
        while true
            μ=2*(rand()-0.5)
            invμ=1/abs(μ)
            l=-log(rand())
            if μ>0
                ΔxQ=x[i]+0.5*Δx[i]-xQ
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ
                shift=1
            elseif μ<0
                ΔxQ=xQ-x[i]+0.5*Δx[i]
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ
                shift=-1
            end
            while l>0
                if i==1 && μ<0 
                    if β1>rand()
                        μ*=-1
                        i=0
                        shift=1
                    else
                        br=true
                        break
                    end  
                elseif i==I && μ>0
                    if β2>rand()
                        μ*=-1
                        i=I+1
                        shift=-1
                    else
                        br=true
                        break
                    end
                end

                i+=shift
                l-=L[i]*invμ
                ϕ2[i]+=invμ*Δx[i]

            end
            if br
                break
            end

            Reactions[i]+=1 
            if μ>0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]+0.5*Δx[i]+ΔxQ
                ϕ2[i]+=invμ*ΔxQ
            elseif μ<0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]-0.5*Δx[i]+ΔxQ
                ϕ2[i]-=invμ*ΔxQ
            end 
            if Σ_t[i]*rand()<Σ_r[i]
                Abs[i]+=1
                break
            end
        end
    end
    ϕ=Sf*Reactions./L/Particles
    ϕ2*=Sf/Particles
    ϕ2./=Δx
    F=reduce(vcat,[νΣ_f[i]*Δx[i]*(ϕ2[i]+ϕ[i])*0.5 for i=1:I])
    K=sum(F)/Sf
    Reactions*=0
    K_vec[batch]=K
    if batch <= inactive
        print("\nBatch : ", batch, " K=", K)
    else
        Kav,stdn=mean(K_vec[inactive+1:batch]),std(K_vec[inactive+1:batch])/sqrt(batch-inactive)
        print("Batch : ", batch, " K=", Kav," ± ",stdn," Error is ",(Kav-4.2300)/stdn, " σ\n")
    end
    Fp=copy(F)/K
    Sf=sum(Fp)
    Qp=Fp/Sf
end


Batch : 1 K=3.981608198126114
Batch : 2 K=4.120844353289832
Batch : 3 K=4.187385884449004
Batch : 4 K=4.200276509744034
Batch : 5 K=4.219398138139733
Batch : 6 K=4.218050989048179
Batch : 7 K=4.228850756817717
Batch : 8 K=4.237383136408599
Batch : 9 K=4.226027317933152
Batch : 10 K=4.224890904803467
Batch : 11 K=4.235042564530305
Batch : 12 K=4.24664358981556
Batch : 13 K=4.2239751978271025
Batch : 14 K=4.218955184209843
Batch : 15 K=4.23571374842482
Batch : 16 K=4.231846936849946
Batch : 17 K=4.221713619982259
Batch : 18 K=4.234426532759301
Batch : 19 K=4.238467175473083
Batch : 20 K=4.234762052449271Batch : 21 K=4.234653627409162 ± NaN Error is NaN σ
Batch : 22 K=4.231109593702463 ± 0.003544033706698091 Error is 0.3130877960798409 σ
Batch : 23 K=4.234682461248765 ± 0.004117293707621329 Error is 1.137266753668043 σ
Batch : 24 K=4.232936257679736 ± 0.003394890372589216 Error is 0.8649050064894882 σ
Batch : 25 K=4.231843055326268 ± 0.0028478517817416464 Error is 0.647173893698925 σ
Bat

In [72]:
plot(K_vec)
plot!(ones(100)*4.23)

In [96]:
K_error=zeros(80)
for i=21:100
    K_error[i-20]=abs(mean(K_vec[21:i])-4.2300)
end
plot(K_error)

In [55]:
f_l=1/0.415
r_l=1/0.371
#Defining geometry
function geometry(x)
    f_l=1/0.415
    r_l=1/0.371
    
    one=(x>0) && (x<r_l)
    two=(x>r_l) && (x<(f_l+r_l))
    three=(x>(f_l+r_l)) && (x<(f_l+(2*r_l)))
    four=(x>(f_l+(2*r_l))) && (x<((2*f_l)+(2*r_l)))
    five=(x>((2*f_l)+(2*r_l))) && (x<((2*f_l)+(3*r_l)))
    six=(x>((2*f_l)+(3*r_l))) && (x<((3*f_l)+(3*r_l)))
    seven=(x>((3*f_l)+(3*r_l))) && (x<((3*f_l)+(4*r_l)))
    
    fuel=two || four || six
    reflector=one || three || five || seven
    return fuel,reflector
end

#Assigning cross sections

function Σs(x)
    fuel,reflector=geometry(x)
    return (fuel*0.334)+(reflector*0.334)
end
    
function Σt(x)
    fuel,reflector=geometry(x)
    return (fuel*0.415)+(reflector*0.371)
end
        
function νΣf(x)
    fuel,reflector=geometry(x)
    return (fuel*0.178)+(reflector*0)
end

Σr(x)=Σt(x)-Σs(x)
β1=0   #Reflective
β2=0   #Vacuum
        
widths=[r_l,f_l,r_l,f_l,r_l,f_l,r_l]
NΔx=[1,1,1,1,1,1,1]*125

n=widths,NΔx ;

In [56]:
Δx=Float64[]
for i=1:length(widths)
    Δx=[Δx;ones(NΔx[i]).*widths[i]/NΔx[i]]
end
I=sum(NΔx)
x=[sum(Δx[1:i])-*(Δx[i]/2) for i=1:I]
Reactions=zeros(I);
Abs=copy(Reactions)
L1,L2=0,0;
νΣ_f=reduce(vcat,[νΣf(x[i]) for i=1:I])
Σ_t=reduce(vcat,[Σt(x[i]) for i=1:I])
Σ_r=reduce(vcat,[Σr(x[i]) for i=1:I])
Σ_s=reduce(vcat,[Σs(x[i]) for i=1:I])
L=reduce(vcat,[Σt(x[i])*Δx[i] for i=1:I]);
Ls=reduce(vcat,[Σs(x[i])*Δx[i] for i=1:I])
Lr=reduce(vcat,[Σr(x[i])*Δx[i] for i=1:I])
ϕ=ones(I);
ϕ2=zeros(I)
Kp=1
Fp=reduce(vcat,[νΣ_f[i]*Δx[i]*ϕ[i] for i=1:I])/Kp
Sf=sum(Fp)
Qp=Fp/Sf;
Pa=Σ_r./Σ_t;

In [43]:
Batches=100
inactive=20
K_vec=zeros(Batches);
@time for batch=1:Batches
    ϕ2*=0
    Particles=10000
    for particle=1:Particles
        pQ=rand()

        i=0
        while pQ>0
            i+=1
            pQ-=Qp[i]    
        end

        xQ=x[i]+(0.5+pQ/Qp[i])*Δx[i] 

        br=false
        while true
            μ=2*(rand()-0.5)
            invμ=1/abs(μ)
            l=-log(rand())
            if μ>0
                ΔxQ=x[i]+0.5*Δx[i]-xQ
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ
                shift=1
            elseif μ<0
                ΔxQ=xQ-x[i]+0.5*Δx[i]
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ
                shift=-1
            end
            while l>0
                if i==1 && μ<0 
                    if β1>rand()
                        μ*=-1
                        i=0
                        shift=1
                    else
                        br=true
                        break
                    end  
                elseif i==I && μ>0
                    if β2>rand()
                        μ*=-1
                        i=I+1
                        shift=-1
                    else
                        br=true
                        break
                    end
                end

                i+=shift
                l-=L[i]*invμ
                ϕ2[i]+=invμ*Δx[i]

            end
            if br
                break
            end

            Reactions[i]+=1 
            if μ>0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]+0.5*Δx[i]+ΔxQ
                ϕ2[i]+=invμ*ΔxQ
            elseif μ<0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]-0.5*Δx[i]+ΔxQ
                ϕ2[i]-=invμ*ΔxQ
            end 
            if Σ_t[i]*rand()<Σ_r[i]
                Abs[i]+=1
                break
            end
        end
    end
    ϕ=Sf*Reactions./L/Particles
    ϕ2*=Sf/Particles
    ϕ2./=Δx
    F=reduce(vcat,[νΣ_f[i]*Δx[i]*(ϕ2[i]+ϕ[i])*0.5 for i=1:I])
    K=sum(F)/Sf
    Reactions*=0
    K_vec[batch]=K
    if batch <= inactive
        print("\nBatch : ", batch, " K=", K)
    else
        Kav,stdn=mean(K_vec[inactive+1:batch]),std(K_vec[inactive+1:batch])/sqrt(batch-inactive)
        print("Batch : ", batch, " K=", Kav," ± ",stdn," Error is ",(Kav-1.17361)/stdn, " σ\n")
    end
    Fp=copy(F)/K
    Sf=sum(Fp)
    Qp=Fp/Sf
end


Batch : 1 K=1.1755263061903471
Batch : 2 K=1.1540661439916198
Batch : 3 K=1.1571289393034534
Batch : 4 K=1.1682847810664727
Batch : 5 K=1.1570991238876667
Batch : 6 K=1.1795994374695813
Batch : 7 K=1.1869644332709028
Batch : 8 K=1.1643729510293366
Batch : 9 K=1.1779224667937358
Batch : 10 K=1.1635774577478553
Batch : 11 K=1.1813265156231576
Batch : 12 K=1.170504567751565
Batch : 13 K=1.1582575346913546
Batch : 14 K=1.1683248026916067
Batch : 15 K=1.1797288329939335
Batch : 16 K=1.1561838702216003
Batch : 17 K=1.1575719363876602
Batch : 18 K=1.1781737843790603
Batch : 19 K=1.1991304526404543
Batch : 20 K=1.171200258375417Batch : 21 K=1.1766133333091173 ± NaN Error is NaN σ
Batch : 22 K=1.1717609029833542 ± 0.004852430325762991 Error is -0.3810661653045101 σ
Batch : 23 K=1.164659724432288 ± 0.0076338345652485345 Error is -1.1724481964092186 σ
Batch : 24 K=1.1621521221722333 ± 0.005951956332116677 Error is -1.9250608015956356 σ
Batch : 25 K=1.1634695725810575 ± 0.004794908350257105 Error

Eigenvalue calculation with survival biasing

In [57]:
Batches=100
inactive=20
K_vec=zeros(Batches);
@time for batch=1:Batches
    Particles=10000
    ϕ2*=0

    for j=1:Particles
        pQ=rand()

        i=0
        while pQ>0
            i+=1
            pQ-=Qp[i]    
        end

        xQ=x[i]+(0.5+pQ/Qp[i])*Δx[i] 
        ω=1
        br=false
        while ω>0
            if ω<0.25
                if rand()<ω
                    ω=1
                    #print("I am alive xD")
                else
                    ω=0
                    #print("Save me I am dying!\n")
                    continue
                end
            end
            μ=2*(rand()-0.5)
            invμ=1/abs(μ)
            l=-log(rand())
                
            if μ>0
                ΔxQ=x[i]+0.5*Δx[i]-xQ
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ*ω
                shift=1
            elseif μ<0
                ΔxQ=xQ-x[i]+0.5*Δx[i]
                l-=ΔxQ*Σ_t[i]*invμ
                ϕ2[i]+=invμ*ΔxQ*ω
                shift=-1
            end
            while l>0
                if i==1 && μ<0 
                    if β1>rand()
                        μ*=-1
                        i=0
                        shift=1
                    else
                        br=true
                        break
                    end  
                elseif i==I && μ>0
                    if β2>rand()
                        μ*=-1
                        i=I+1
                        shift=-1
                    else
                        br=true
                        break
                    end
                end

                i+=shift
                l-=L[i]*invμ
                ϕ2[i]+=invμ*Δx[i]*ω

            end
            if br
                break
            end

            
            if μ>0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]+0.5*Δx[i]+ΔxQ
                ϕ2[i]+=invμ*ΔxQ*ω
            elseif μ<0
                ΔxQ=l*μ/Σ_t[i]
                xQ=x[i]-0.5*Δx[i]+ΔxQ
                ϕ2[i]-=invμ*ΔxQ*ω
            end
            
            Abs[i]+=ω*Pa[i]
            ω*=1-Pa[i]
            Reactions[i]+=ω
        end
    end
    ϕ=Sf*(Reactions+Abs)./L/Particles;
    ϕ2*=Sf/Particles
    ϕ2./=Δx
    
    F=νΣ_f.*Δx.*(ϕ2+ϕ)*0.5
    K=sum(F)/Sf
    Reactions*=0
    Abs*=0
    K_vec[batch]=K
    if batch <= inactive
        print("\nBatch : ", batch, " K=", K)
    else
        Kav,stdn=mean(K_vec[inactive+1:batch]),std(K_vec[inactive+1:batch])/sqrt(batch-inactive)
        print("Batch : ", batch, " K=", Kav," ± ",stdn," Error is ",(Kav-1.17361)/stdn, " σ\n")
    end
    Fp=copy(F)/K
    Sf=sum(Fp)
    Qp=Fp/Sf
end


Batch : 1 K=1.1524468887375445
Batch : 2 K=1.1658281447045782
Batch : 3 K=1.1611688797472701
Batch : 4 K=1.1660488204452009
Batch : 5 K=1.1664367134094136
Batch : 6 K=1.170141572828526
Batch : 7 K=1.1718710093104225
Batch : 8 K=1.1737311519941485
Batch : 9 K=1.1673713543205357
Batch : 10 K=1.171513673785504
Batch : 11 K=1.1676907793542775
Batch : 12 K=1.1677807665329634
Batch : 13 K=1.169855847524286
Batch : 14 K=1.1799026392114087
Batch : 15 K=1.1772012405205463
Batch : 16 K=1.1732987935416899
Batch : 17 K=1.1612355109983503
Batch : 18 K=1.171016542257041
Batch : 19 K=1.181256930186445
Batch : 20 K=1.1834701923350563Batch : 21 K=1.176793288979444 ± NaN Error is NaN σ
Batch : 22 K=1.1809250721491065 ± 0.004131783169662562 Error is 1.7704395048648793 σ
Batch : 23 K=1.1776810266646365 ± 0.0040267077320079265 Error is 1.0110062451953543 σ
Batch : 24 K=1.1784166869521435 ± 0.002940813431369158 Error is 1.6344753124667364 σ
Batch : 25 K=1.1751731816121098 ± 0.003963503131536685 Error is 0.

In [ ]:
widths=[4.863392, 5.630757]

NΔx=[10,10]

N=widths,NΔx ;

#Defining Geometry
function geometry(x,widths=widths)
    i=0
    while x>=0
        i+=1
        x-=widths[i]

    end
    if i==1
        return true,false
    elseif i==2
        return false,true
    else
        return 0,0
    end
    
end

#assigning material properties to the geometry
function Σt(x)
    U235,H2O=geometry(x)
    return U235* [0.88721 ; 2.9727] + H2O * [0.88798 ; 2.9865]
end

function Σr(x)
    U235,H2O=geometry(x)
    return U235*[0.04829 ; 0.0544] + H2O*[0.04823 ; 0.0189]
end

function Σso(x)
    U235,H2O=geometry(x)
    return U235* [0 0.000767 ; 0.04635 0] + H2O * [0 0.000336 ; 0.04749 0]
end

function νΣf(x)
    U235,H2O=geometry(x)
    return U235* [2.09e-3  0.07391]
end

χ(x)=geometry(x)[1]*[1 ; 0]

#Albedo Boundary Conditions
β1=1   #Reflective
β2=1  #Vacuum

bc=β1,β2;

In [110]:
#Defining Geometry
function geometry(x)
    U235 = x>=0 && x<=0.0329074
    H2O = x>=0.0329074 && x<=9.067695
    return U235,H2O
end

#assigning material properties to the geometry
function Σt(x)
    U235,H2O=geometry(x)
    return U235* [0.650917 ; 2.13800] + H2O * [0.1106832906 ; 0.36355]
end

function Σr(x)
    U235,H2O=geometry(x)
    return U235*[0.650917 ; 0.0692] + H2O*[0.001009076 ; 1.6e-4]
end

function Σso(x)
    U235,H2O=geometry(x)
    return U235* [0 0 ; 0.0342008 0] + H2O * [0 0 ; 0.001000595707 0]
end

function νΣf(x)
    U235,H2O=geometry(x)
    return U235* [0.617209  0.11426]
end

D(x)=1/3/Σt(x)
χ(x)=geometry(x)[1]*[1 ; 0]

f=D,Σr,Σso,νΣf,χ

#Albedo Boundary Conditions
β1=1   #Reflective
β2=1   #Vacuum

bc=β1,β2;

widths=[ 0.0329074, 9.034787]
NΔx=[1,9]

N=widths,NΔx ;

In [111]:
Groups=length(χ(0))
Δx=Float64[]
for i=1:length(widths)
    Δx=[Δx;ones(NΔx[i]).*widths[i]/NΔx[i]]
end
I=sum(NΔx)
x=[sum(Δx[1:i])-*(Δx[i]/2) for i=1:I]
Reactions=zeros(I,Groups);
Abs=copy(Reactions)
νΣ_f=map(νΣf,x)
Σ_t=reduce(hcat,map(Σt,x))
Σ_r=reduce(hcat,map(Σr,x))
Σ_so=map(Σso,x)
χ_=reduce(hcat,map(χ,x))
L=reduce(hcat,[Σ_t[:,i].*Δx[i] for i=1:I])
ϕ=ones(I,Groups);
ϕ2=zeros(I,Groups)
Kp=1
F=zeros(I,Groups)
Q_=zeros(I,Groups)
for i=1:I
    F[i,:]=νΣ_f[i]*ϕ[i,:]*Δx[i]/Kp.*χ_[:,i]
    Q_[i,:]=F[i,:]+Σ_so[i]*ϕ[i,:]*Δx[i]
end
Sf=sum(F)
Qn=sum(Q_)
Qp=Q_/Qn
Pa=Σ_r./Σ_t;

In [112]:
Batches=100
inactive=20
K_vec=zeros(Batches);
@time for batch=1:Batches
    Particles=10000
    ϕ2*=0
    Reactions*=0
    Abs*=0
    for j=1:Particles
        pQ=rand()

        i,group=0,1
        while pQ>0
            if i==I
                group+=1
                i=0
            end
            i+=1
            pQ-=Qp[i,group]   
        end

        xQ=x[i]+(0.5+pQ/Qp[i,group])*Δx[i] 
        ω=1
        br=false
        while ω>0
            if ω<0.25
                if rand()<ω
                    ω=1
                else
                    ω=0
                    continue
                end
            end
            μ=2*(rand()-0.5)
            invμ=1/abs(μ)
            l=-log(rand())
                
            if μ>0
                ΔxQ=x[i]+0.5*Δx[i]-xQ
                l-=ΔxQ*Σ_t[group,i]*invμ
                ϕ2[i,group]+=invμ*ΔxQ*ω
                shift=1
            elseif μ<0
                ΔxQ=xQ-x[i]+0.5*Δx[i]
                l-=ΔxQ*Σ_t[group,i]*invμ
                ϕ2[i,group]+=invμ*ΔxQ*ω
                shift=-1
            end
            while l>0
                if i==1 && μ<0 
                    if β1>rand()
                        μ*=-1
                        i=0
                        shift=1
                    else
                        br=true
                        break
                    end  
                elseif i==I && μ>0
                    if β2>rand()
                        μ*=-1
                        i=I+1
                        shift=-1
                    else
                        br=true
                        break
                    end
                end

                i+=shift
                l-=L[group,i]*invμ
                ϕ2[i,group]+=invμ*Δx[i]*ω

            end
            if br
                break
            end

            
            if μ>0
                ΔxQ=l*μ/Σ_t[group,i]
                xQ=x[i]+0.5*Δx[i]+ΔxQ
                ϕ2[i,group]+=invμ*ΔxQ*ω
            elseif μ<0
                ΔxQ=l*μ/Σ_t[group,i]
                xQ=x[i]-0.5*Δx[i]+ΔxQ
                ϕ2[i,group]-=invμ*ΔxQ*ω
            end
            
            Abs[i,group]+=ω*Pa[group,i]
            ω*=1-Pa[group,i]
            Reactions[i,group]+=ω
        end
    end
    for i=1:I
        ϕ[i,:]=Qn*(Reactions+Abs)[i,:]./L[:,i]/Particles
    end
    ϕ2*=Qn/Particles
    ϕ2./=Δx
    ϕ+=ϕ2
    ϕ.*=0.5

    
    for i=1:I
        F[i,:]=νΣ_f[i]*ϕ[i,:]*Δx[i].*χ_[:,i]
    end
    K=sum(F)/Sf
    F./=K
    for i=1:I
        Q_[i,:]=F[i,:]+Σ_so[i]*ϕ[i,:]*Δx[i]
    end
    K_vec[batch]=K
    if batch <= inactive
        print("\nBatch : ", batch, " K=", K)
    else
        Kav,stdn=mean(K_vec[inactive+1:batch]),std(K_vec[inactive+1:batch])/sqrt(batch-inactive)
        print("\nBatch : ", batch, " K=", Kav," ± ",stdn," Error is ",(Kav-1)/stdn, " σ")
    end
    Sf=sum(F)
    Qn=sum(Q_)
    Qp=Q_/Qn
end


Batch : 1 K=1.102744378191949
Batch : 2 K=0.9967316772780462
Batch : 3 K=0.9963838046666381
Batch : 4 K=1.0062565545844488
Batch : 5 K=0.9937628086346456
Batch : 6 K=0.998437763576633
Batch : 7 K=0.9962117453645273
Batch : 8 K=0.9927165310865786
Batch : 9 K=0.9987547954301063
Batch : 10 K=0.9997240787617909
Batch : 11 K=1.0116274630238251
Batch : 12 K=0.9985617540594381
Batch : 13 K=1.0019015847562949
Batch : 14 K=1.010885101549174
Batch : 15 K=1.0034732527306913
Batch : 16 K=1.0045232406617122
Batch : 17 K=1.0088801390803186
Batch : 18 K=0.9955011432062947
Batch : 19 K=1.003967372611125
Batch : 20 K=1.0032521844228734
Batch : 21 K=0.9934388697715801 ± NaN Error is NaN σ
Batch : 22 K=1.0025271975558137 ± 0.009088327784233517 Error is 0.2780706875689333 σ
Batch : 23 K=1.0035916497324908 ± 0.0053540289260508985 Error is 0.6708312155384638 σ
Batch : 24 K=1.004289304244053 ± 0.003849614875282731 Error is 1.1142164561949917 σ
Batch : 25 K=1.0026584954365405 ± 0.003398714195024253 Error is 

In [126]:
widths=[4.863392, 5.630757]

NΔx=[10,10]

N=widths,NΔx ;

#Defining Geometry
function geometry(x,widths=widths)
    i=0
    while x>=0
        i+=1
        x-=widths[i]

    end
    if i==1
        return true,false
    elseif i==2
        return false,true
    else
        return 0,0
    end
    
end

#assigning material properties to the geometry
function Σt(x)
    U235,H2O=geometry(x)
    return U235* [0.88721 ; 2.9727] + H2O * [0.88798 ; 2.9865]
end

function Σr(x)
    U235,H2O=geometry(x)
    return U235*[0.04829 ; 0.0544] + H2O*[0.04823 ; 0.0189]
end

function Σso(x)
    U235,H2O=geometry(x)
    return U235* [0 0.000767 ; 0.04635 0] + H2O * [0 0.000336 ; 0.04749 0]
end

function νΣf(x)
    U235,H2O=geometry(x)
    return U235* [2.09e-3  0.07391]
end

χ(x)=geometry(x)[1]*[1 ; 0]

#Albedo Boundary Conditions
β1=1   #Reflective
β2=0  #Vacuum

bc=β1,β2;

In [127]:
Groups=length(χ(0))
Δx=Float64[]
for i=1:length(widths)
    Δx=[Δx;ones(NΔx[i]).*widths[i]/NΔx[i]]
end
I=sum(NΔx)
x=[sum(Δx[1:i])-*(Δx[i]/2) for i=1:I]
Reactions=zeros(I,Groups);
Abs=copy(Reactions)
νΣ_f=map(νΣf,x)
Σ_t=reduce(hcat,map(Σt,x))
Σ_r=reduce(hcat,map(Σr,x))
Σ_so=map(Σso,x)
χ_=reduce(hcat,map(χ,x))
L=reduce(hcat,[Σ_t[:,i].*Δx[i] for i=1:I])
ϕ=ones(I,Groups);
ϕ2=zeros(I,Groups)
Kp=1
F=zeros(I,Groups)
Q_=zeros(I,Groups)
for i=1:I
    F[i,:]=νΣ_f[i]*ϕ[i,:]*Δx[i]/Kp.*χ_[:,i]
    Q_[i,:]=F[i,:]+Σ_so[i]*ϕ[i,:]*Δx[i]
end
Sf=sum(F)
Qn=sum(Q_)
Qp=Q_/Qn
Pa=Σ_r./Σ_t;

In [128]:
Batches=100
inactive=20
K_vec=zeros(Batches);
@time for batch=1:Batches
    Particles=100000
    ϕ2*=0
    Reactions*=0
    Abs*=0
    for j=1:Particles
        pQ=rand()

        i,group=0,1
        while pQ>0
            if i==I
                group+=1
                i=0
            end
            i+=1
            pQ-=Qp[i,group]   
        end

        xQ=x[i]+(0.5+pQ/Qp[i,group])*Δx[i] 
        ω=1
        br=false
        while ω>0
            if ω<0.25
                if rand()<ω
                    ω=1
                else
                    ω=0
                    continue
                end
            end
            μ=2*(rand()-0.5)
            invμ=1/abs(μ)
            l=-log(rand())
                
            if μ>0
                ΔxQ=x[i]+0.5*Δx[i]-xQ
                l-=ΔxQ*Σ_t[group,i]*invμ
                ϕ2[i,group]+=invμ*ΔxQ*ω
                shift=1
            elseif μ<0
                ΔxQ=xQ-x[i]+0.5*Δx[i]
                l-=ΔxQ*Σ_t[group,i]*invμ
                ϕ2[i,group]+=invμ*ΔxQ*ω
                shift=-1
            end
            while l>0
                if i==1 && μ<0 
                    if β1>rand()
                        μ*=-1
                        i=0
                        shift=1
                    else
                        br=true
                        break
                    end  
                elseif i==I && μ>0
                    if β2>rand()
                        μ*=-1
                        i=I+1
                        shift=-1
                    else
                        br=true
                        break
                    end
                end

                i+=shift
                l-=L[group,i]*invμ
                ϕ2[i,group]+=invμ*Δx[i]*ω

            end
            if br
                break
            end

            
            if μ>0
                ΔxQ=l*μ/Σ_t[group,i]
                xQ=x[i]+0.5*Δx[i]+ΔxQ
                ϕ2[i,group]+=invμ*ΔxQ*ω
            elseif μ<0
                ΔxQ=l*μ/Σ_t[group,i]
                xQ=x[i]-0.5*Δx[i]+ΔxQ
                ϕ2[i,group]-=invμ*ΔxQ*ω
            end
            
            Abs[i,group]+=ω*Pa[group,i]
            
            ω*=1-Pa[group,i]
            Reactions[i,group]+=ω
        end
    end
    for i=1:I
        ϕ[i,:]=Qn*(Reactions+Abs)[i,:]./L[:,i]/Particles
    end
    ϕ2*=Qn/Particles
    ϕ2./=Δx
    ϕ+=ϕ2
    ϕ.*=0.5

    
    for i=1:I
        F[i,:]=νΣ_f[i]*ϕ[i,:]*Δx[i].*χ_[:,i]
    end
    K=sum(F)/Sf
    F./=K
    for i=1:I
        Q_[i,:]=F[i,:]+Σ_so[i]*ϕ[i,:]*Δx[i]
    end
    K_vec[batch]=K
    if batch <= inactive
        print("\nBatch : ", batch, " K=", K)
    else
        Kav,stdn=mean(K_vec[inactive+1:batch]),std(K_vec[inactive+1:batch])/sqrt(batch-inactive)
        print("Batch : ", batch, " K=", Kav," ± ",stdn," Error is ",(Kav-1)/stdn, " σ\n")
    end
    Sf=sum(F)
    Qn=sum(Q_)
    Qp=Q_/Qn
end


Batch : 1 K=0.9921612056093216
Batch : 2 K=0.9853497621406285
Batch : 3 K=0.9856731818799601
Batch : 4 K=0.9994604626209878
Batch : 5 K=0.984242361987639
Batch : 6 K=0.9939282936871818
Batch : 7 K=1.0028230920520347
Batch : 8 K=0.9996883490533726
Batch : 9 K=0.9974224223455698
Batch : 10 K=0.9990103692700101
Batch : 11 K=0.9968258013258177
Batch : 12 K=0.992702712136691
Batch : 13 K=1.0000212740530114
Batch : 14 K=1.0070089027705118
Batch : 15 K=0.9971552560324104
Batch : 16 K=1.0051797659615975
Batch : 17 K=0.9944071603637034
Batch : 18 K=1.0128980013523754
Batch : 19 K=0.9940121159835493
Batch : 20 K=0.9967880889364067Batch : 21 K=1.0029841626611786 ± NaN Error is NaN σ
Batch : 22 K=1.0014772752822134 ± 0.0015068873789652604 Error is 0.9803488321919752 σ
Batch : 23 K=1.0000284533637525 ± 0.0016899669057039424 Error is 0.016836639614906786 σ
Batch : 24 K=0.9987678415628752 ± 0.0017369905537158226 Error is -0.7093639251456454 σ
Batch : 25 K=0.9982787922604102 ± 0.001431590350086363 Er

In [129]:
plot(x,ϕ)